In [9]:
import pandas as pd
import lxml.etree as etree
import re
import numpy as np
from after_preprocess_function import split_strength
from after_preprocess_function import reorder_dfs

In [2]:
tree = etree.parse("data_preprocessed/full database.xml")
namespace = {"db" : "http://www.drugbank.ca"}

In [4]:
all_drugs = []
drug_synonyms = []
for drug in tree.findall("db:drug", namespace):
    #all_products = []
    for products in drug.find("db:products", namespace):
        #all_products.append([
        drug_name = drug.find("db:name", namespace).text,
        synonyms = [syn.text for syn in drug.find("db:synonyms", namespace)],
        product = products.find("db:name", namespace).text,
        country = products.find("db:country", namespace).text,
        strength = products.find("db:strength", namespace).text,
        route = products.find("db:route", namespace).text,
        #    ])
        all_drugs.append([drug_name, product, country, strength, route])
        drug_synonyms.append([drug_name, synonyms])

In [5]:
matches = []
for a in drug_synonyms:
    for b in a[1][0]:
        matches.append([a[0][0],b])

In [6]:
mylist = []
for list in all_drugs:
    sublist = []
    for item in list:
        sublist.append(item[0])
    mylist.append(sublist)

In [7]:
df_drug_bank = pd.DataFrame(mylist, 
columns = ["drug","product","country", "strength", "route"])

In [10]:
df_drug_bank[['strength','unit']] = pd.DataFrame(df_drug_bank['strength'].apply(split_strength).tolist(), index=df_drug_bank.index)
df_drug_bank = reorder_dfs(df_drug_bank)

In [12]:
df_drug_bank.to_csv('data_after_preprocess/data_drug_bank.csv', index=False)

In [18]:
synonyms = pd.DataFrame(matches,columns=['drug','synonym']).drop_duplicates()

In [19]:
synonyms['drug'] = synonyms['drug'].str.upper()
synonyms['synonym'] = synonyms['synonym'].str.upper()

In [20]:
synonyms.to_csv('data_after_preprocess/synonyms.csv', index=False)